# Inspector Introduction

**In this notebook we introduce the Inspector component of HERE Data SDK for Python and demonstrate advanced styling with its ipyleaflet-based backend.**

**Features:**
* Show simple usage patterns related to the inspector.
* Show more advanced customization use cases.
* Show ipyleaflet-specific styling options

**Dependencies:**
* Sample GeoJSON data, obtained from the freely available content on [GitHub](https://github.com/johan/world.geo.json) and available in the sample_datasets folder provided with this notebook
* Catalog: [HERE GeoJSON Samples](https://platform.here.com/data/hrn:here:data::olp-here:here-geojson-samples/overview)
* Languages: Python

## Sample datasets

Load datasets from various sources that will be shown using the inspector.

In [ ]:
import json
import geopandas as gpd
from shapely.geometry import shape

In [ ]:
with open("sample_datasets/DEU.geo.json") as f:
    data1 = json.load(f) # this is a GeoJSON FeatureCollection with only Germany

ger = data1["features"][0] # this is the GeoJSON Feature of Germany

In [ ]:
with open("sample_datasets/FRA.geo.json") as f:
    data2 = json.load(f) # this is a GeoJSON FeatureCollection with only France

fra = data2["features"][0] # this is the GeoJSON Feature of France

In [ ]:
with open("sample_datasets/IRL.geo.json") as f:
    data3 = json.load(f) # this is a GeoJSON FeatureCollection with only Ireland

irl = data3["features"][0] # this is the GeoJSON Feature of Ireland

In [ ]:
# This is a GeoJSON FeatureCollection with Germany and France
data4 = {
    "type": "FeatureCollection",
    "features": [ger, fra]
}

In [ ]:
with open("sample_datasets/countries.geo.json") as f:
    data5 = json.load(f) # this is a GeoJSON FeatureCollection with all the countries of the world

In [ ]:
rows = [
    {
        "id": feat["id"], 
        "name": feat["properties"]["name"], 
        "geometry": shape(feat["geometry"])
    } for feat in data5["features"] if feat["properties"]["name"].lower().startswith("po")
]

gdf1 = gpd.GeoDataFrame.from_records(rows, index="id").set_geometry("geometry")
gs1 = gdf1.geometry

gdf1 # This is a GeoDataFrame with Poland and Portugal

Loading higher resolution country polygons from a HERE platform catalog

In [ ]:
from here.platform import Platform
from here.geopandas_adapter import GeoPandasAdapter

In [ ]:
platform = Platform(adapter=GeoPandasAdapter())
sample_catalog = platform.get_catalog("hrn:here:data::olp-here:here-geojson-samples")
borders_layer = sample_catalog.get_layer("country-borders")

In [ ]:
gdf2 = borders_layer.read_partitions(partition_ids=[1419, 1422])
gs2 = gdf2.geometry

gdf2 # This is a GeoDataFrame with some country polygons

In [ ]:
# Example shapely geometries: points, linestrings and polygons

from shapely.geometry import shape, Point, MultiPoint, LineString, Polygon

pt1 = Point(30, 50)
pt2 = Point(32, 49)
pt3 = Point(31, 52)

ls1 = LineString([(29, 49), (30, 51), (31, 49), (32, 50), (33, 51)])
ls2 = LineString([(30, 47), (31, 48), (29, 49), (31, 50), (29, 51)])

poly1 = Polygon([(28, 46), (28, 52), (29, 53), (31, 53), (33, 48), (32, 47), (28, 46)])
poly2 = MultiPoint([(40, 51), (43, 49)]).buffer(3)

## Inspecting datasets

The inspector is able to visualize data in different formats. Supported are:

- `gpd.GeoSeries`
- `gpd.GeoDataFrame`
- a python container, e.g. list, of shapely `BaseGeometry` objects, such as `Point`, `LineString` and `Polygon`
- one GeoJSON `FeatureCollection`, as python dictionary
- a python container, e.g. list, of GeoJSON `Feature`, each as python dictionary

Geographic features may be inspected singly or layered.
Each layer may have its own color or custom style.

Inspector can be used in 3 ways:

1. In one line, through the simplified `inspect` function.
2. Instantiating an implementation of `Inspector`, such as `IpyleafletInspector` and configuring its properties.
3. Accessing the underlaying inspector implementation via the `backend` function,
   for `IpyleaftletInspector` this exposes the `ipyleaflet.Map` object that the user
   can further customize to leverage all the capabilities of _ipyleaflet_.

For more information, please refer to the documentation of the `inspect` function and `Inspector` interface.

### 1. The `inspect` function and basic styling

This function opens an inspector and loads geodata in a single call. It can be used to quicly inspect the data, if no further customizations are needed.

You can visualize one or more datadasets. Each dataset may have a name and a style. Names and styles are default empty. Default style is a color automatically assigned by the inspector. You can specify a name and/or a different color, or a more complex rendering style as explained in the second part.

If you have a HERE API key, you can set it as shown here. The inspect works also without an API key, but specifying a key gives you access to HERE base maps and HERE content by default.

In [ ]:
from here.inspector import options

options.api_key = None # Set your HERE API key to use the HERE tiles by default

For more information about `options` and the options available, please see the section below in this notebook.

#### Empty inspectors

In [ ]:
from here.inspector import inspect

inspect()

In [ ]:
# With different HERE tiles
from here.inspector import here_basemaps, IpyleafletInspector

here_inspector = IpyleafletInspector().set_basemap(here_basemaps.terrainDay)
here_inspector.show()

In [ ]:
# Centered in Buenos Aires

inspect(center=Point(-58.381667, -34.603333), zoom=10)

#### GeoSeries and GeoDataFrame

In [ ]:
# Inspect one single GeoDataFrame

inspect(gdf1) # just the content

In [ ]:
# Inspect one single GeoSeries

inspect(gs2, "An example GeoSeries") # content and name

In [ ]:
# Inspect multiple GeoSeries and GeoDataFrames, as independent layers

inspect(layers={"A": gdf1, "B": gs2}) # content with names, default-styled

#### GeoJSON

In [ ]:
# Inspect one single GeoJSON FeatureCollection
from here.inspector.styles import Color

inspect(data4, "Central EU", Color.BLUE) # content, name and style

In [ ]:
# Inspect one single GeoJSON Feature

inspect(fra, style=Color.RED) # unnamed, just style

#### Shapely geometries

In [ ]:
# Inspect more shapely geometries in one layer

inspect([pt1, pt2, ls1, poly1]) # unnamed, default-styled

In [ ]:
# Inspect shapely geometries in multiple layer

inspect(layers={
    "Points and lines A": [pt1, ls1],
    "Points and lines B": [pt2, pt3, ls2],
    "Polygons": [poly1, poly2]
}) # named, each layer default-styled

#### Tiling grid

A tiling grid can be added to every inspector which will draw the borders of the HEREtile partitions given. You must specify which tiles to display by providing a list or `Series` of HEREtile partition IDs.

In [ ]:
# Selected tiles

tiles = [5763, 5766, 5761]

inspect(tiles=tiles)

In [ ]:
# Display features along with the tiles existing inside bounding box
import pandas as pd
from here.geotiles.heretile import in_bounding_box

features = [
    Point(-100, 30).buffer(10),
    LineString([(-110, 30), (-100, 50)]).buffer(2)
]
tiles = pd.Series(in_bounding_box(west=-110, south=35, east=-80, north=50, level=7))

inspect(features, tiles=tiles, tiles_style=Color.ORANGE)

#### Mixed layer types and tiling

In [ ]:
# Inspect more than one layer
# each layer can be of different type, in this example
# GeoDataFrame, GeoSeries, collection of shapely geometric data

inspect(layers={
    "Example GeoDataFrame": gdf1,
    "Example GeoSeries": gs2,
    "Example GeoJSON Feature": irl,
    "Example GeoJSON FeatureCollection": data4,
    "Example geometries": [pt1, pt2, ls1, ls2, poly2]
}, layers_style={
    "Example geometries": Color.GRAY
}) # named, each layer default-styled, but geometries

## Example analysis

Show the top 10 partitions containing countries with the largest borders geometry (by data size).

This is a quick analysis performed solely on metadata. Content is downloaded to extract
country polygons and show them on a map.

In [ ]:
# Get the metadata of the sample layer

mdata = borders_layer.get_partitions_metadata()

In [ ]:
mdata.sort_values(by="data_size", ascending=False)[:10]

In [ ]:
large_ids = mdata.sort_values(by="data_size", ascending=False)[:10].id.values
gdf = borders_layer.read_partitions(partition_ids=list(large_ids))
gdf

In [ ]:
# Inspect the result

inspect(gdf, "10 largest partitions", Color.PINK)

### 2. The `Inspect` interface

You can create an inspector and fine-tune the visualization by configuring the inspector with various functions. The customization possibilities, although extended, are similar to the one provided by the `inspect` function.

You can visualize one or more datadasets and one or more tiling grids. Each appaear as layer, and layer may have a name, style, or more complex rendering style.

#### Generic styling

In [ ]:
# Instantiate the default inspector
from here.inspector import new_inspector

inspector = new_inspector()

# Load some data, each creates a layer
inspector.add_features(fra) # unnamed content
inspector.add_features(ger, style=Color.YELLOW)  # unnamed content with style
inspector.add_features(gs2, name="Some mediterranean countries", style=Color.BLUE) # named and styled

# Further configuration
inspector.set_zoom(4)

# Show the inspector once configuration is complete
inspector.show()

#### Multiple tiling grids

In [ ]:
# Instantiate a default inspector
from here.geotiles.heretile import in_geometry

inspector = new_inspector()

# Load some data, each creates a layer
inspector.add_features(fra, name="France", style=Color.BLUE)

# Define some tiles
tiles1 = pd.Series(in_bounding_box(west=-10, south=30, east=5, north=40, level=6))
tiles2 = pd.Series(in_geometry(Point(10, 45).buffer(8), level=8, fully_contained=True))

# Add the two tiling grids
inspector.add_tiles(tiles1, name="Tiling grid, level 6", style=Color.GREEN)
inspector.add_tiles(tiles2, name="Tiling grid, level 8", style=Color.YELLOW)

# Show the inspector once configuration is complete
inspector.show()

#### _ipyleaflet_-specific styles

Styling possibilities are not limited to generic styles and colors. Each inspector implementation supports implementation-specific styles.

For the case of the _ipyleaflet_-based inspector, `style` dictionaries as described in [ipyleaflet documentation](https://ipyleaflet.readthedocs.io/)
can be passed in place of generic styles. This includes support for `hover_style` and `point_style` as well.

A method `set_basemap` is provided to easily configure a custom _ipyleaflet_ [base map](https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html).

In [ ]:
# This example requires explicit use of the ipyleaflet-based implementation
from here.inspector.ipyleaflet import IpyleafletInspector
from here.inspector import external_basemaps

# Instantiate the ipyleaflet-based inspector
inspector = IpyleafletInspector()

# Add features with ipyleaflet-specific styles
inspector.add_features(
    fra, name="France",
    style={'color': 'cyan', 'radius': 8, 'fillColor': '#cc6633', 'opacity': 0.5, 'weight': 5, 'dashArray': '10', 'fillOpacity': 0.4},
    hover_style={'fillColor': 'cyan', 'fillOpacity': 0.4}
)
inspector.add_features(
    [pt1, pt2, pt3, ls1, poly2], name="Some geometries",
    style={'color': 'magenta', 'fillColor': 'white', 'opacity': 0.8, 'weight': 3, 'dashArray': '5', 'fillOpacity': 0.8},
    hover_style={'fillColor': 'yellow', 'fillOpacity': 0.4},
    point_style={'radius': 10}
)

# Set a different base map
inspector.set_basemap(external_basemaps.Esri.WorldImagery)

# Show the inspector once configuration is complete
inspector.show()

### 3. The `backend` function to access implementation details

Via the `Inspector.backend` function it's possible to access the underlying rendering backend. This enables access to all the advanced functionalities of the rendering backend for more advanced use cases and unlimited customization. The implementation is provided already configured with theme, features, and tiling grids specified so far. Users can add content.

For `IpyleafletInspector`, this provides access to a preconfigured _ipyleaflet_ `Map` object. This in turns enables defining custom layer types, further map content, and UI widgets. For more information, please see the _ipyleaftlet_ [documentation](https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html).

## Inspector Options: default themes and colors

The inspector is configurable by changing the properties in `here.inspector.options`, that are used as default values. These include default theme and styles, also also HERE API key to enable base maps provided by HERE.

In [ ]:
print(options)

Multiple default themes and colors are provided in the inspector: 

In [ ]:
from here.inspector.styles import Theme

for theme in Theme:
    print(theme)

print("Default:", options.default_theme)

In [ ]:
for color in Color:
    print(color)

print("Default:", [c.name for c in options.default_colors])

The default theme is used to style the map, including the base map, and define RGB values for each color. The default colors are cycled through by the inspector to select which color to use for features and tiling grids in case no style or color is specified. Apart from setting specific themes, colors and other settings using methods of `Inspector`, the user can override these default values directly in the options (shown here only for the theme):

In [ ]:
# From this moment on, all the inspector instances use dark theme
options.default_theme = Theme.DARK_MAP

inspect()

It's also possible to specify a different theme or sequence of colors using the methods `set_theme` and `set_colors` of `Inspector`.

In [ ]:
# This resets the default to `LIGHT_MAP` to not interfere with the cells below
options.default_theme = Theme.LIGHT_MAP

### Blank themes

A default map is added as base map in the default theme. It's possible to use themes ending with `_BLANK` to avoid displaying a base map.

In [ ]:
# Instantiate the default inspector
inspector = new_inspector()

# Load some data, each creates a layer
inspector.add_features(fra, "France", Color.BLUE)
inspector.add_features(ger, "Germany", Color.RED)
inspector.add_features(irl, "Ireland", Color.GREEN)
inspector.add_features(gs2, "Country mix", Color.GRAY)

# Configure the theme
inspector.set_theme(Theme.LIGHT_BLANK)

# Show the inspector once configuration is complete
inspector.show()

### Dark themes

Use themes beginning with `DARK_` for a more comfortable experience at night or dark environements. RGB values of standard colors are adjusted to the theme.

In [ ]:
# Instantiate the default inspector
inspector = new_inspector()

# Load some data, each creates a layer
inspector.add_features(fra, "France", Color.BLUE)
inspector.add_features(ger, "Germany", Color.RED)
inspector.add_features(irl, "Ireland", Color.GREEN)
inspector.add_features(gs2, "Country mix", Color.GRAY)

# Configure the theme
inspector.set_theme(Theme.DARK_MAP)

# Show the inspector once configuration is complete
inspector.show()

### Examples of base maps
Here inspector supports multiple ways to define base maps:

- HERE base maps
- External base maps
- Custom base maps

HERE base maps and External base maps are `Tileprovider` objects provided by [xyzservices](https://github.com/geopandas/xyzservices) package. For HERE base maps you need to add your HERE API key to the global inspector options written directly in the code or taken from an environment variable. 

### Example of base maps from HERE

In case a HERE API key is available, the inspector takes advantage of it authenticate MapTile API.  
Just specify your own key in `here.inspector.options.api_key` as shown at the beginning of the notebook. HERE maps are visualised if the HERE API key is set, otherwise fall-back base maps are used. Below example shows all the supported HERE base maps.

In [ ]:
here_basemaps

In [ ]:
from IPython.display import display

if options.api_key:
    inspector = IpyleafletInspector()
    inspector.set_basemap(basemap=here_basemaps.normalDay)
    inspector.set_zoom(14)
    inspector.set_center(Point(13.4083, 52.5186))
    display(inspector.show())

### Example of base maps from external providers
External base maps are XYZ tile providers other than HERE.

In [ ]:
external_basemaps

In [ ]:
inspector = IpyleafletInspector()
inspector.set_basemap(basemap=external_basemaps.OpenStreetMap.Mapnik)
inspector.show()

Some external basemaps need an auth token/api key.
You can check if external basemap requires authentication token using `requires_token()` method on Tileprovider.
If token/api key is required then user need to set API key given by provider in basemap object.

In [ ]:
if external_basemaps.MapBox.requires_token():
    external_basemaps.MapBox["accessToken"] = "YOUR-PERSONAL-TOKEN"

### Example of a custom base map

In [ ]:
inspector = IpyleafletInspector()

basemap = {
    'name': 'U.S. Geological Survey',
    'url': 'https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}',
    'maxZoom': 20,
    'attribution': 'Tiles courtesy of the <a href="https://usgs.gov/">U.S. Geological Survey</a>'
}

inspector.set_basemap(basemap)
inspector.show()

<!--
from urllib.parse import quote, unquote
svg = """<svg width="40" height="48" viewBox="0 0 40 48" fill="none" xmlns="http://www.w3.org/2000/svg"><path class="triangle" d="M11,36.8l-5.5,5.5L0,36.8H11z" fill="#48DAD0"></path><path class="HERE" d="M19.1,24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9,1.7-0.5,2.7,0.5L19.1,24.2z M30.9,9.2c0.9-0.9,1.7-0.5,2.7,0.5
    L31.3,12C30.1,10.7,30.2,9.9,30.9,9.2z M38,11.6c-1.1,1.6-2.9,4.2-4.9,2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4
	c-1.6,1.6-2,3.4-1.5,5.1l-1.6-1.8c-0.5,0.3-2.4,1.9-0.9,4.5l-1.8-1.5l-2.4,2.4l3.2,3.2c-2.5-1.9-5.1-1.8-7.2,0.4
	c-2.3,2.3-2.1,5-0.4,7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9,0.9-1.5,2.2-1.3,3.1L4,24.6l-2.6,2.6l9.6,9.6h5.2l-3.5-3.5
	c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8,1.8-0.3,3.6,1.4l3.4,3.4l2.6-2.6l-3.2-3.2c2.3,1.8,5.1,1.9,7.7-0.6l0,0c1.5-1.4,2-2.8,2-2.8
	l-1.9-1.3c-1.1,1.6-2.9,4.2-4.9,2.3l5-5l3.1,3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5,0-4.1c0.4,0.7,0.9,1.5,1.5,2.1
	c2.5,2.5,5.7,3,8.6,0.2l0,0c1.5-1.4,2-2.8,2-2.8S38,11.6,38,11.6z" fill="#000000"></path></svg>
"""
print(f"""![HERE](data:image/svg+xml,{quote(svg)})
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>""")
-->
![HERE](data:image/svg+xml,%3Csvg%20width%3D%2240%22%20height%3D%2248%22%20viewBox%3D%220%200%2040%2048%22%20fill%3D%22none%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20class%3D%22triangle%22%20d%3D%22M11%2C36.8l-5.5%2C5.5L0%2C36.8H11z%22%20fill%3D%22%2348DAD0%22%3E%3C/path%3E%3Cpath%20class%3D%22HERE%22%20d%3D%22M19.1%2C24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5L19.1%2C24.2z%20M30.9%2C9.2c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5%0A%20%20%20%20L31.3%2C12C30.1%2C10.7%2C30.2%2C9.9%2C30.9%2C9.2z%20M38%2C11.6c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4%0A%09c-1.6%2C1.6-2%2C3.4-1.5%2C5.1l-1.6-1.8c-0.5%2C0.3-2.4%2C1.9-0.9%2C4.5l-1.8-1.5l-2.4%2C2.4l3.2%2C3.2c-2.5-1.9-5.1-1.8-7.2%2C0.4%0A%09c-2.3%2C2.3-2.1%2C5-0.4%2C7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9%2C0.9-1.5%2C2.2-1.3%2C3.1L4%2C24.6l-2.6%2C2.6l9.6%2C9.6h5.2l-3.5-3.5%0A%09c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8%2C1.8-0.3%2C3.6%2C1.4l3.4%2C3.4l2.6-2.6l-3.2-3.2c2.3%2C1.8%2C5.1%2C1.9%2C7.7-0.6l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8%0A%09l-1.9-1.3c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.3l5-5l3.1%2C3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5%2C0-4.1c0.4%2C0.7%2C0.9%2C1.5%2C1.5%2C2.1%0A%09c2.5%2C2.5%2C5.7%2C3%2C8.6%2C0.2l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8S38%2C11.6%2C38%2C11.6z%22%20fill%3D%22%23000000%22%3E%3C/path%3E%3C/svg%3E%0A)
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>